In [11]:
import re
import numpy as np
import sys
import time
d_corpus = sc.textFile("../data/SmallTrainingData.txt", 1)
d_keyAndText = d_corpus.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('">') + 2:][:-6]))
regex = re.compile('[^a-zA-Z]')
d_keyAndListOfWords = d_keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

In [12]:
allWords = d_keyAndListOfWords.flatMap(lambda x: x[1]).map(lambda x: (x, 1))
from operator import add
allCounts = allWords.reduceByKey(add)

In [13]:
allCounts.take(5)

[('consideration', 336),
 ('of', 205873),
 ('an', 23896),
 ('pending', 64),
 ('appeal', 879)]

In [14]:
topWords = allCounts.top(20000, key=lambda x: x[1])

In [15]:
twentyK = sc.parallelize(range(20000))

In [16]:
dictionary = twentyK.map (lambda x : (topWords[x][0], x))

In [17]:
pairApplicant = dictionary.filter(lambda x: x[0] == "applicant").collect()

In [18]:
pairAnd = dictionary.filter(lambda x: x[0] == "and").collect()
pairAttack = dictionary.filter(lambda x: x[0] == "attack").collect()
pairProtein= dictionary.filter(lambda x: x[0] == "protein").collect()
pairCar = dictionary.filter(lambda x: x[0] == "car").collect()

In [9]:
print("Position of applicant: ", pairApplicant[0][1])
print("Position of and: ", pairAnd[0][1])
print("Position of attack: ", pairAttack[0][1])
print("Position of protein: ", pairProtein[0][1])
print("Position of car: ", pairCar[0][1])

Position of applicant:  346
Position of and:  2
Position of attack:  502
Position of protein:  3014
Position of car:  608


In [10]:
#task 2

In [19]:
def buildArray (listOfIndices):
    returnVal = np.zeros (20000)
    for index in listOfIndices:
        returnVal[index] = returnVal[index] + 1
    mysum = np.sum (returnVal)
    returnVal = np.divide (returnVal, mysum)
    return returnVal


In [20]:
allWords = d_keyAndListOfWords.flatMap(lambda x: ((j, x[0]) for j in x[1]))

In [21]:
allDictionaryWords = dictionary.join(allWords)

In [22]:
justDocAndPos = allDictionaryWords.map(lambda x: (x[1][1], x[1][0]))

In [23]:
allDictionaryWordsInEachDoc = justDocAndPos.groupByKey()

In [24]:
forCSV= allDictionaryWordsInEachDoc.mapValues(lambda x: np.array( list(x)))

In [25]:
tfs = allDictionaryWordsInEachDoc.map(lambda x: (x[0], buildArray(x[1])))

In [24]:
tfs.take(5)

[('AU35',
  array([0.11425061, 0.06511057, 0.02272727, ..., 0.        , 0.        ,
         0.        ])),
 ('AU85',
  array([0.11214496, 0.0553379 , 0.03770813, ..., 0.        , 0.        ,
         0.        ])),
 ('AU160',
  array([0.08426966, 0.04283708, 0.02457865, ..., 0.        , 0.        ,
         0.        ])),
 ('AU561',
  array([0.09657795, 0.03802281, 0.01977186, ..., 0.        , 0.        ,
         0.        ])),
 ('AU736',
  array([0.11322314, 0.0446281 , 0.01983471, ..., 0.        , 0.        ,
         0.        ]))]

In [25]:
print(re.compile('^AU').match('AU35'))

<_sre.SRE_Match object; span=(0, 2), match='AU'>


In [26]:
def oneHotEncoding(ID):
    if re.compile('^AU').match(ID):
        return 1
    else:
        return 0

In [27]:
data = tfs.map(lambda x: (oneHotEncoding(x[0]), x[1]))
        #.map(lambda x: (x[0], np.append(1.0, x[1])))

In [43]:
data.take(5)

[(1, array([0.11425061, 0.06511057, 0.02272727, ..., 0.        , 0.        ,
         0.        ])),
 (1, array([0.11214496, 0.0553379 , 0.03770813, ..., 0.        , 0.        ,
         0.        ])),
 (1, array([0.08426966, 0.04283708, 0.02457865, ..., 0.        , 0.        ,
         0.        ])),
 (1, array([0.09657795, 0.03802281, 0.01977186, ..., 0.        , 0.        ,
         0.        ])),
 (1, array([0.11322314, 0.0446281 , 0.01983471, ..., 0.        , 0.        ,
         0.        ]))]

In [28]:
def calculateTheta(x, r_prev):
    return np.dot(x, r_prev)

In [29]:
def getLogPart(theta):
    return np.log(1 + np.exp(theta))

In [30]:
def calculateXtY(x, y):
    return -np.multiply(x, y)

In [31]:
def getXMultiplyThetaPart(x, theta):
    return np.multiply(x, (np.exp(theta) / (1.0 + np.exp(theta))))

In [32]:
def getYMultiplyTheta(y, theta):
    return -np.multiply(y, theta)

In [33]:
def getCostAndDerivative(x, y, r_prev):
    theta = calculateTheta(x, r_prev)
    Ytheta = getYMultiplyTheta(y, theta)
    logPart = getLogPart(theta)
    XtY = calculateXtY(x, y)
    XTheta = getXMultiplyThetaPart(x, theta)
    cost = np.add(Ytheta, logPart)
    derivative = np.add(XtY, XTheta)
    return np.array([cost, derivative])

In [50]:
def getDerivative(x, y, theta):
    XtY = calculateXtY(x, y)
    XTheta = getXMultiplyThetaPart(x, theta)
    derivative = np.add(XtY, XTheta)
    return derivative

In [51]:
def getLLH(x, y, theta):
    Ytheta = getYMultiplyTheta(y, theta)
    logPart = getLogPart(theta)
    cost = np.add(Ytheta, logPart)
    return cost

In [ ]:
#def gradientDescent(learningRate = 0.01, num_iteration = 100):
num_iteration = 100
learningRate = 0.01
r_prev= np.full(20000, 0.1)
count = 0
cost_prev = sys.float_info.max
precision = 0.00000001
prev_stepsize = 1
while(num_iteration > count and precision < prev_stepsize):
    start = time.time()

    # Calculate cost and derivative
    costAndDerivative = data.map(lambda x: (x[0], x[1], calculateTheta(x[1], r_prev)))\
                            .map(lambda x: (1, np.append(getLLH(x[1], x[0], x[2]), getDerivative(x[1], x[0], x[2]))))\
                            .reduceByKey(lambda x1, x2:np.add(x1, x2)).collect()
   

    cost_cur = costAndDerivative[0][1][0]
    dr = costAndDerivative[0][1][1:]


    # Set current weight 
    r_cur = np.subtract(r_prev, learningRate*dr)



    # Bold Driver logic
    if cost_cur < cost_prev:
        learningRate *= 1.05
    else:
        learningRate *= 0.5

    print("Iteration: ", count)
    print("W: " ,r_cur)
    print("Loss: ", cost_cur)

    count += 1

    # Calculate stepsize
    prev_stepsize = np.linalg.norm(np.subtract(r_prev, r_cur), 2)

    r_prev = r_cur

    cost_prev = cost_cur

    stop = time.time()
    duration = stop - start
    print("Duration", duration)

In [ ]:

prediction = 